<a href="https://colab.research.google.com/github/GGMmattos/PIC/blob/main/R3_TOLD_BR_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Importing needed libraries
!pip install -U scikit-learn
!pip install -U spacy
!python -m spacy download pt


!pip install wandb
!pip install --upgrade simpletransformers
!pip install --upgrade transformers

from pandas.core.window.ewm import OnlineExponentialMovingWindow
import pandas as pd
import numpy as np
import seaborn as sns #Para visualização de gráficos
import matplotlib.pyplot as plt #Para visualização de gráficos
import plotly.express as px #Biblioteca para geração de gráfico dinámico
%matplotlib inline
from collections import Counter
import re, nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity ='all'
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd
import logging
import numpy as np
import sklearn
import pickle
import spacy
from transformers import AutoTokenizer  # Or BertTokenizer
from transformers import AutoModelForPreTraining  # Or BertForPreTraining for loading pretraining heads
from transformers import AutoModel  # or BertModel, for BERT without pretraining heads
from transformers import pipeline
from sklearn.model_selection import train_test_split

In [ ]:
told_br = pd.read_csv("/content/drive/MyDrive/PIC /Dataset/told_binary_2_colums.csv")
base_hatebr = pd.read_csv('/content/drive/MyDrive/PIC /Dataset/HateBR.csv')

In [ ]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)



# Preparing train data
base_toldbr_train_data  = told_br.iloc[:,[0, 1]]
base_hatebr_eval_data = base_hatebr.iloc[:,[0, 1]]

train_df = pd.DataFrame({
    'text': base_toldbr_train_data['text'],
    'label': base_toldbr_train_data['toxic']
})

eval_df = pd.DataFrame({
    'text': base_hatebr_eval_data['instagram_comments'],
    'label': base_hatebr_eval_data['offensive_language']
})


#Optional model configuration
model_args = ClassificationArgs(
  num_train_epochs=1,
  overwrite_output_dir=True
                                )
#Create a ClassificationModel
model = ClassificationModel(
    "bert", "neuralmind/bert-base-portuguese-cased", args=model_args,
    use_cuda=False
)

model.train_model(train_df)

# Save model
with open('TOLD_BR-BERT_FULL.pkl ', 'wb') as file:
    pickle.dump(model, file)

In [ ]:
with open('/content/drive/MyDrive/PIC /Models/TOLD_BR-BERT_FULL.pkl', 'rb') as f:
    model = pickle.load(f)

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(eval_df)
lst = []
for arr in model_outputs:
    lst.append(np.argmax(arr))
true = eval_df['label'].tolist()
predicted = lst
print(classification_report(true, predicted))

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:1454: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/7000 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/875 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0       0.72      0.95      0.82      3500
           1       0.93      0.62      0.75      3500

    accuracy                           0.79      7000
   macro avg       0.82      0.79      0.78      7000
weighted avg       0.82      0.79      0.78      7000

